In [25]:
import random
import math


class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False

    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table



Hash Table With Count

In [24]:
class HashTableWithCount():

    def __init__(self, initial_capacity=(100)): # We're gonna need 100 entries for the tests
        self.table_size = self.get_next_prime(initial_capacity) # make a table sized w a prime over 100
        self.table = [None] * self.table_size                   # fill it with None
        self.item_count = 0     # a variable to track how many elements are added/removed
        self.total_probes = 0   # a variable to track probes for the experiment

    def get_next_prime(self, n):
        # uhh, let's use our existing prime methods
        candidate = (n+1)                   # starting with the next possible number
        while not self.is_prime(candidate): # check if it's prime
            candidate += 1                  # if not, iterate
        return candidate                    # return the next prime found

    def isOdd(self, n): # a helper function to check if an integer is odd
        if((n%2)==0):
            return False
        else:
            return True
        
    # thank heck we made one of these earlier
    def is_prime(self, n):
        if(n==1 or n==0):       # handles the cases of 1 and 0
            return False
        elif(n==2):             # handles the case of 2
            return True
        elif(not self.isOdd(n)):     # handles all even numbers except 0 and 2
            return False
        for x in range(2,round(n**(1/2))+1):  # for every number x between 2 and sqroot n
            if(self.isOdd(x)):                   # only checks odds cause we already checked 2
                if((n%x)==0):               # if n divides by x evenly
                    return False            # return false
        return True                         # if none divide evenly, it's a prime
    
    # def check_size(self, size): 
    # above is the original header; didn't end up using size though
    def check_size(self):         # i'm not sure why size is passed to this
        size = self.table_size          # size is table size
        ratio = (self.item_count/size)  # we get a ratio of how full the array is
        return ratio                    # return the ratio

    def get_hash_index(self, key):
        # computes the first index of the given key
        return (hash(key) % self.table_size)

    def is_hash_table_too_full(self):
        if(self.check_size() > 0.7):  # checks if the hash table is 70% full
            return True             # returns True, it is to full, if over 0.7
        else: return False          # returns false if check returns 0.7 or less

# might throw an error if not called by the subclasses, cause the super doesn't have "add"
    def enlarge_hash_table(self):
        # makes the table size the next prime number at least double the current size
        new_table_size = (self.get_next_prime(self.table_size*2))
        old_table = self.table
        self.table = [None] * new_table_size
        self.table_size = new_table_size
        self.item_cout = 0 # reset the item count before adding items to the new table

        for entry in old_table:
            if entry is not None and entry.is_in_table():
                self.add(entry.get_key(), entry.get_value())

    def reset_probe_count(self):
        self.total_probes = 0 # just sets probes to 0

    def get_probes(self):
        return self.total_probes # returns the probe total

table = HashTableWithCount()

# testing enlarge function
#print(table.table_size)
#table.enlarge_hash_table()
#print(table.table_size)
#table.enlarge_hash_table()
#print(table.table_size)

#print(table.is_prime(20))
#print(table.isOdd(5))



Get Statistics

In [46]:
import random   # we're allowed to import random and math right?
import math     # both of these get imported in TableEntry, so I'm just going to assume

class GetStatistics:

    @staticmethod
    def main():
        linear_ht = LinearProbingWithCount()
        double_ht = DoubleHashingWithCount()

        linear_count = []   # list for linear prob stats
        dhash_count = []    # list for double hash stats

        names = GetStatistics.generate_names()  # a list to hold 11k three letter names
        add_list = GetStatistics.get_1000_names(names)
        search_list = GetStatistics.get_10000_names(names)

        for t in range(1000):
            GetStatistics.choose_100_names_and_add(linear_ht, double_ht, add_list) # add names from add list
            GetStatistics.search_for_100_names(linear_ht,double_ht,search_list)    # search names
            linear_count.append(linear_ht.get_probes())              # add probe count from-
            dhash_count.append(double_ht.get_probes())               # this test iteration

        print("linear_count: ", linear_count) # print our probe count lists
        print("dhash_count: ", dhash_count)

        GetStatistics.compute_statistics(linear_count,dhash_count)
        
        
        

    # i got tunnel vision and made the goofiest name generator possible
    # as a result we need an extra method to bodge things together
    @ staticmethod
    def generate_names():   # going to generate 676 (26^2) for the first list
                            # then grab the 11k after those, so they're all 3 character strings
        geny = name_generator(676,11676) 
        return geny[1]      # skip the 2 letter ones, grab the list of 3 letter names
    
    @staticmethod
    def choose_100_names_and_add(linear_ht, double_ht, add_list):
        # randomly chooses 100 names from the 1000
        # adds em to both linear_ht and double_ht
        one_hundred_names = random.sample(add_list, 100)    # grab a random sample of names
        for name in one_hundred_names:  # for each name in our random sample
            linear_ht.add(name,name)    # add to linear_ht
            double_ht.add(name,name)    # add to double_ht
        linear_ht.reset_probe_count()   # reset the probe count cause adding might use probes
        double_ht.reset_probe_count()   # do the same for double_ht

    @staticmethod
    def get_1000_names(names):
        return names[:1000] # grab the first 1000 names from list
        
    @staticmethod
    def get_10000_names(names):
        return names[1000:] # grab the next 10k names from list

    @staticmethod
    def search_for_100_names(linear_ht,double_ht,search_list): # changed names to search_list here
        one_hundred_names = random.sample(search_list, 100)
        for name in one_hundred_names:
            linear_ht.locate(name)  # search linear
            double_ht.locate(name)  # search double
        # i guess we can grab our probe counts
    
    @staticmethod
    def compute_statistics(linear_count, dhash_count):
        linear_mean = sum(linear_count) / len(linear_count)
        double_mean = sum(dhash_count) / len(dhash_count)       # get our means and print them

        linear_diffs = [(i - linear_mean) ** 2 for i in linear_count]   
        double_diffs = [(i - double_mean) ** 2 for i in dhash_count]    

        l_variance = sum(linear_diffs) / (len(linear_count) - 1)
        d_variance = sum(double_diffs) / (len(dhash_count) -1)

        linear_standard_dev = math.sqrt(l_variance)
        double_standard_dev = math.sqrt(d_variance)

        print("linear hash count mean: ", linear_mean)
        print("linear hash standard deviation: ", linear_standard_dev)
        print("double hash count mean: ", double_mean)
        print("double hash standard deviation: ", double_standard_dev)
        pass
        

if __name__ == "__main__":
    GetStatistics.main()




linear_count:  [50, 38, 57, 34, 40, 60, 32, 57, 29, 27, 27, 29, 20, 20, 33, 39, 29, 36, 34, 41, 60, 37, 44, 52, 29, 44, 58, 46, 37, 28, 44, 42, 51, 38, 53, 40, 59, 39, 57, 45, 36, 62, 66, 49, 64, 30, 67, 44, 58, 41, 44, 59, 52, 51, 72, 53, 69, 73, 48, 60, 62, 52, 59, 55, 52, 77, 53, 48, 46, 66, 49, 42, 58, 36, 50, 51, 39, 56, 34, 59, 28, 39, 35, 30, 41, 50, 51, 39, 82, 42, 43, 47, 53, 51, 53, 67, 35, 39, 34, 62, 59, 54, 37, 55, 65, 54, 67, 36, 60, 60, 57, 37, 59, 63, 61, 44, 28, 46, 65, 57, 45, 46, 27, 60, 39, 69, 55, 66, 58, 76, 47, 62, 67, 58, 48, 42, 70, 68, 45, 43, 55, 41, 51, 56, 50, 52, 43, 64, 60, 52, 60, 45, 49, 66, 57, 33, 76, 47, 49, 57, 53, 47, 48, 66, 47, 57, 49, 51, 61, 58, 43, 51, 42, 51, 49, 51, 47, 41, 73, 52, 75, 51, 49, 42, 54, 51, 54, 49, 69, 56, 55, 53, 56, 40, 52, 46, 71, 62, 51, 55, 79, 41, 41, 47, 70, 63, 45, 59, 53, 47, 54, 37, 69, 58, 49, 44, 57, 50, 35, 55, 55, 46, 51, 67, 22, 40, 53, 69, 44, 62, 50, 41, 55, 51, 60, 38, 47, 62, 63, 37, 42, 49, 59, 44, 50, 49, 

Experiments:
    initial test results:
        linear hash count mean:  50.408
        linear hash standard deviation:  10.74928291017978
        double hash count mean:  39.857
        double hash standard deviation:  7.694137427337177
    This isn't right. They're supposed to be closer aren't they? I wonder if I'm counting probes wrong somewhere

In [ ]:
# Below is GetStatistics test/diag code that's been cut out

# print("lcount length: ", len(linear_count))
# print("dhcount length: ", len(dhash_count))

Linear Probing With Count

In [ ]:
class LinearProbingWithCount(HashTableWithCount):
    # def probe(self, index, key)
    # above is the old header but I didn't need "key" in my implementation
    def probe(self, index):
        self.total_probes += 1  # increment probe count
        return ((index + 1) % self.table_size) # return the next index
    ##################################################################################
    # def locate(self, index, key): # why do we pass the index up here?
    # above is the old input; but I didn't need to pass index
    def locate(self, key): # why do we pass the index up here?
        index = self.get_hash_index(key) # 
        while self.table[index] is not None:# stop if we hit an empty cell
            if self.table[index].get_key() == key:  # if the key is at this index
                return index                        # return the index it was found in
            index = self.probe(index)       # use probe to search further indices
        return None                         # return none if we don't find the key
    ##################################################################################
    def add(self,key,value):
        index = self.get_hash_index(key)            # grab the index
        while self.table[index] is not None:        # until we hit an empty
            if self.table[index].get_key() == key:  # if it's already in there
                self.table[index].set_value(value)  # update the value
                return                              # don't run forever
            index = self.probe(index)               # otherwise, probe til we find it
        self.table[index] = TableEntry(key,value)   # insert the new entry
        self.item_count += 1                         # increment num_items
        if self.is_hash_table_too_full():   # check if the table is reaching capacity
            self.enlarge_hash_table()       # enlarge if needed
    ##################################################################################
    def remove(self, key):
        index = self.locate(key) # pull the index of the given key using locate()
        if index is not None:           # if the item is found
            self.table[index] = None    # remove the entry
            self.item_count -= 1         # decrement item count
        else:                           # if something goes wrong
            print("Linear remove operation failed: Key not found") # lmk
    ##################################################################################
    def get_value(self, key):
        index = self.locate(key)    # use locate to find the object
        if index is not None:       # if entry is found
            entry = self.table[index]   # grab the found object
            return entry.get_value()    # use TableEntry's get_value method
        return None # If the entry isn't found, return None
    ##################################################################################
    def contains(self, key):
        index = self.locate(key)# locate the index
        if index is not None:   # if we find the key
            return True         # return True
        return False            # else return False
    ##################################################################################
    def is_empty(self):
        return (self.item_count==0) # True if 0 items, False otherwise
    ##################################################################################
    def get_size(self):
        return self.item_count # returns the item count
    ##################################################################################
    def clear(self):
        self.table = [None] * 101   # empty the table, set it to default size
        self.item_count = 0          # reset item count

In [ ]:
# linear probe testing

Double Hashing With Count

In [42]:
class DoubleHashingWithCount(HashTableWithCount):
    def primary_hash(self, key):
        """First hash function."""
        return hash(key) % self.table_size
    ##################################################################################
    def secondary_hash(self, key):
        """Second hash function to calculate step size."""
        return 1 + (hash(key) % (self.table_size - 1))
    ##################################################################################
    def probe(self, index, key):
        # I've set probe up to do the stepping, while Locate and Add will interpret
        # the result returned by probe
        step = self.secondary_hash(key)             # finds the step length
        #index = (index + step) %                   # find the next index
        #self.total_probes += 1                      # add to probe count
        while (self.table[index] is not None) and (self.table[index].get_key() != key):
                                        # until we find the key, or an empty bucket
            self.total_probes += 1                  # iterate probe count
            index = (index + step)%self.table_size  # we keep stepping
        return index        # we return the index of the given key or empty bucket
    ##################################################################################
    # def locate(self, index, key): # don't think i'll need to pass index
    def locate(self, key):
        index = self.primary_hash(key)                  # grabs the first index
        while self.table[index] is not None:            # until we hit an empty
                if self.table[index].get_key() == key:  # if the key is in the bucket
                    return index                        # located, return the index
                index = self.probe(index,key)           # probe and grab the index
        return None                                     # if we hit an empty, return None
    ##################################################################################
    def add(self,key,value):    # same add method as Linear
        index = self.primary_hash(key)              # grab the first index
        while self.table[index] is not None:        # until we hit an empty
            if self.table[index].get_key() == key:         # if it's already in there
                self.table[index].set_value(value)  # update the value
                return                              # don't run forever
            index = self.probe(index,key)           # otherwise, probe til we find it
        self.table[index] = TableEntry(key,value)   # insert the new entry
        self.item_count += 1                         # increment num_items
        if self.is_hash_table_too_full():   # check if the table is reaching capacity
            self.enlarge_hash_table()       # enlarge if needed
    ##################################################################################
    def remove(self, key):          # same remove method should do the trick
        index = self.locate(key)    # pull the index of the given key using locate()
        if index is not None:           # if the item is found
            self.table[index] = None    # remove the entry
            self.item_count -= 1         # decrement item count
        else:                           # if something goes wrong
            print("Double hash remove operation failed: Key not found") # lmk
    ##################################################################################
    def get_value(self, key):           # i'd love to just return key for a giggle
        index = self.locate(key)        # grab the appropriate index using locate
        if index is not None:                   # if we find the item
            return self.table[index].get_value()# return its value
        return None                             # return none if not found
    ##################################################################################
    def contains(self, key):    # same method as Linear should work
        index = self.locate(key)# locate the index
        if index is not None:   # if we find the key
            return True         # return True
        return False            # else return False
    ##################################################################################
    def is_empty(self):             # checks if the table is empty
        return (self.item_count==0)  # returns True if it's empty and False otherwise
    ##################################################################################
    def get_size(self):
        return self.item_count # returns the item count
    ##################################################################################
    def clear(self):                # flips the table over and gets a new one
        self.table = [None] * 101   # empty the table, set it to default size
        self.item_count = 0          # reset item count



In [ ]:
# double hash testing

Name Generator:

In [16]:
def name_generator(n1,n2):   

    genList = []

    v = 1
    while (26**v)<(n1+n2):  # find how many letters we need to satisfy both lists
        v += 1              # increment v (the number of letters we'll use)
        
    while (len(genList) < (n1+n2)):     # till genList is as many elements as called for
        for j in range(2,v+1):          # generate all the names with j letters
            genList.extend(recursy(j))  # til we reach v letters
    
    listN1 = genList[:n1]       # make the first n1 elements listN1
    listN2 = genList[n1:]       # make the rest listN2
    listN2 = listN2[:n2]        # chop the list to n2 length

        # Some diagnostic stuff here
    #print("listN1 length: ",len(listN1)) # for diag
    #print("listN2 length: ",len(listN2)) # for diag
    #print(listN1)
    #print(listN2)
    return (listN1,listN2)

def recursy(lCount = int, returnList = None, rCount = 0, name_string = ""):
    charlist = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    # returns a list of all letter combinations with lCount number of characters
    # recurses to lCount depth
    # each appends their letter to name_string
    if returnList is None:  # if there isn't yet a returnlist
        name_string = ""    # set name string
        returnList = []     # set return list
    

    for c in charlist: # for every char in the alphabet
        # append the char to the passed name_string
        new_string = name_string + c
        # iterate rCount
        rCount = rCount+1

        if (rCount == lCount): # if it's the deepest instance of recursion
            returnList.append(new_string)
            # name_string = "" # reset string
            
            # the word should be complete now, so we add it to the return list
        else:   # if it's not the deepest instance
                # call recursy again
            recursy(lCount, returnList, rCount, new_string)
        rCount -= 1 # decrement rCount after recursion
    # once all the looping mumbo is done
    return returnList # return the list

    
# print(recursy(lCount=2)) # testing
# print(name_generator(100,1000)) # testrun
# name_generator(10,100)

Below is the blooper reel

In [ ]:
"""
        found_key = False
        while not found_key:           # until we find the key
            if(self.table[index][0]==key):  # check if our key matches the key in that index
                found_key = True            # if it does, end loop
            elif(self.table[index] == None):# if we hit an empty
                return -1                   # -1, item not found
            else:                           # else
                # modulo get_size lets us wrap back around if we hit the end of the array
                index = (index+1) % self.get_size# increment index
        return index                        # return the index if found
"""
# When this project started, I tried for a whole different name generator that would make
# names that actually made sense rather than random strings, but it ended up being too
# convoluted and I had to trash it


# print("EEEEEEEEEEEEEEEeeeeeeeeeeeeeeeEEEEEEEEEEEEEEE")